In [43]:
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
import pytorch_lightning as pl
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import random_split
from torchvision.transforms import RandomHorizontalFlip, RandomVerticalFlip
import matplotlib.pyplot as plt
import numpy as np
import torch.nn.functional as F
import pandas as pd
import os

## Define parameters

In [44]:
learning_rate = 1e-3
input_size = 100
output_size = 100
batch_size = 1
num_epochs = 2
num_workers = 8
train_dir = os.path.join(os.getcwd(), 'data', 'train')
test_dir = os.path.join(os.getcwd(), 'data', 'test')
dtype = torch.float32

## Define the neural network architecture
Layer options (More info at https://pytorch.org/docs/stable/nn.html):
+ Linear: fully connected layer
+ Conv1d/Conv2d: Convolutional layers
+ BatchNorm2d/LayerNorm/InstanceNorm2d: Normalization layers
+ Dropout: Dropout layer
+ MaxPool2d/AvgPool2d: Pooling layers

In [45]:
class NN(pl.LightningModule):
    def __init__(self, learning_rate, input_size, output_size):
        super(NN, self).__init__()
        self.learning_rate = learning_rate
        self.layer1 = nn.Linear(input_size, 2*91)
        self.layer2 = nn.Linear(2*91, 91)
        self.relu = nn.ReLU()
        self.loss_fn = nn.MSELoss()
    
    def forward(self, x):
        x = self.relu(self.layer1(x))
        x = self.layer2(x)
        return x

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.learning_rate)
    

    def training_step(self, batch, batch_idx):
        inputs, targets = batch
        outputs = self.forward(inputs)
        loss = self.loss_fn(outputs, targets)
        return loss
    
    def test_step(self, batch, batch_idx):
        inputs, targets = batch
        outputs = self.forward(inputs)
        loss = self.loss_fn(outputs, targets)
        return loss



## Custom dataset class (TODO)

In [46]:
class CustomDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        # Assuming each item in the dataset is a tuple of (input, output)
        sample = self.data[index]
        input_array, output_array = sample[0], sample[1]

        # Convert to PyTorch tensors
        input_tensor = torch.Tensor(input_array, dtype=dtype)
        output_tensor = torch.Tensor(output_array, dtype=dtype)

        return input_tensor, output_tensor
    
class CustomDataModule(pl.LightningDataModule):
    def __init__(self, batch_size, num_workers):
        super().__init__()
        self.batch_size = batch_size
        self.num_workers = num_workers

    def prepare_data(self):
        pass

    def setup(self, stage):
        train_data = []
        test_data = []
        for subdir in os.listdir(train_dir):
            input_output = (pd.read_csv(os.path.join(train_dir, subdir, 'polar.csv')).values.transpose(), pd.read_csv(os.path.join(train_dir, subdir, 'coords.csv')).values.transpose())
            train_data.append(input_output)
            break
        for subdir in os.listdir(test_dir):
            input_output = (pd.read_csv(os.path.join(test_dir, subdir, 'polar.csv')).values.transpose(), pd.read_csv(os.path.join(test_dir, subdir, 'coords.csv')).values.transpose())
            test_data.append(input_output)
            break

        self.train_ds = CustomDataset(train_data)
        self.test_ds = CustomDataset(test_data)

    def train_dataloader(self):
        return DataLoader(
            self.train_ds,
            batch_size=self.batch_size,
            #num_workers=self.num_workers,
        )

    def test_dataloader(self):
        return DataLoader(
            self.test_ds,
            batch_size=self.batch_size,
            #num_workers=self.num_workers,
        )

In [47]:
class DistanceCallback(pl.Callback):
    def __init__(self, dataset):
        super().__init__()
        self.dataset = dataset

    def on_epoch_end(self, trainer, pl_module):
        if trainer.current_epoch == trainer.max_epochs - 1:
            # Calculate distances for the last epoch
            distances = self.calculate_distances(pl_module, self.dataset)
            
            # Plot average and maximum distances
            self.plot_distances(distances)

    def calculate_distances(self, pl_module, dataset):
        pl_module.eval()
        distances = []

        for data in DataLoader(dataset, batch_size=1):
            inputs, targets = data
            with torch.no_grad():
                outputs = pl_module(inputs)
            distances.append(self.calculate_distance(outputs, targets))

        return distances

    def calculate_distance(self, output, target):
        # Replace this with your distance calculation method
        # Here, I'm using L2 distance for illustration purposes
        distance = torch.norm(output - target)
        return distance.item()

    def plot_distances(self, distances):
        avg_distance = np.mean(distances)
        max_distance = np.max(distances)

        plt.figure(figsize=(8, 5))
        plt.hist(distances, bins=50, color='skyblue', edgecolor='black')
        plt.axvline(avg_distance, color='red', linestyle='dashed', linewidth=2, label=f'Average Distance: {avg_distance:.2f}')
        plt.axvline(max_distance, color='green', linestyle='dashed', linewidth=2, label=f'Max Distance: {max_distance:.2f}')
        plt.title('Distance Distribution')
        plt.xlabel('Distance')
        plt.ylabel('Frequency')
        plt.legend()
        plt.show()

In [48]:
# Set device cuda for GPU if it's available otherwise run on the CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
lightning_model = NN(learning_rate=learning_rate, input_size=input_size, output_size=output_size).to(device)


dm = CustomDataModule(batch_size, num_workers)

# Train and plot loss over epochs, as well as average and maximum difference for every sample in the last epoch

In [50]:
callbacks = [pl.callbacks.LearningRateMonitor(logging_interval='epoch'),
             pl.callbacks.ProgressBar(),
             DistanceCallback(dataset=dm)]
trainer = pl.Trainer(max_epochs=num_epochs, callbacks=callbacks, log_every_n_steps=1)  # Adjust parameters as needed

# Train the model
trainer.fit(lightning_model, dm)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name    | Type    | Params
------------------------------------
0 | layer1  | Linear  | 546   
1 | layer2  | Linear  | 16.7 K
2 | relu    | ReLU    | 0     
3 | loss_fn | MSELoss | 0     
------------------------------------
17.2 K    Trainable params
0         Non-trainable params
17.2 K    Total params
0.069     Total estimated model params size (MB)
c:\Users\Marco\AppData\Local\Programs\Python\Python310\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:432: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
c:\Users\Marco\AppData\Local\Programs\Python\Python310\lib\site-package

RuntimeError: The size of tensor a (91) must match the size of tensor b (2) at non-singleton dimension 2

# Get results from testing:


In [ ]:
trainer.test(lightning_model, dm)
